In [67]:
# https://www.youtube.com/watch?v=jMU9G5WEtBc&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=15
# 미분식 : https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1

In [71]:
# 1. 직접 미분해서 구해보기
# 시그모이드는 사용안된다 
# softmax -> log -> 취해줌
# log_softmax -> 취해주고 다 더해주고 평균

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim 

x_data = [[1],[2],[3],[11],[12],[13],[21],[22],[23]]
y_data = [[0],[0],[0],[1], [1], [1], [2], [2], [2]]

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

w = torch.zeros((1,3))
b = torch.zeros(1)

learning_rate = 0.01
epochs = 1000

for epoch in range(1000):
    for tmp_idx in range(len(x_data)):
        now_train_x = x_train[tmp_idx]
        now_train_y = y_train[tmp_idx]  
        
        before_softmax = now_train_x@w + b 
        
        # print(before_softmax)
        after_softmax =  torch.softmax(before_softmax, dim =0)
        after_log = -torch.log(after_softmax)
        
        # print(f"x_train : {now_train_x}")
        # print(f"y_train : {now_train_y}")
        # print(f"after softmax : {after_softmax}")
        # print(f"after log : {after_log}")


        hypothesis = after_log
        y_one_hot = torch.zeros_like(hypothesis)
        y_one_hot.scatter_(0, now_train_y ,1)
        
        # print(f"y_one_hot :  {y_one_hot}")
        
        cost = (y_one_hot * after_log).sum()
        loss_derivate_wrt_to_z = after_softmax - y_one_hot
        # print(f"loss_derivate : {loss_derivate_wrt_to_z}")
        
        gradient_w = (loss_derivate_wrt_to_z) * now_train_x
        gradient_b = loss_derivate_wrt_to_z.mean()
        # print(f"gradient_w : {gradient_w}")
        # print(f"gradient_b : {gradient_b}")        
        
        w -= learning_rate * gradient_w
        b -= learning_rate * gradient_b
    if epoch % 100 == 0:
        print(f'Epoch ({str(epoch).rjust(4)}/1000) cost: {round(float(cost),6)}, W: {w}, b:{round(float(b),7)}')
    

print(f'W: {w}')
print(f'b: {b}')
print('result : ')
print(torch.softmax(x_train@w + b,dim=1))    

# 결과
# 가장 초기버전 
# wx + b 로는 표현하기 좀 힘든거같음
# 직접 미분, SGD 로는 조금 불안정한걱 같음 
    

Epoch (   0/1000) cost: 0.01824, W: tensor([[-0.0665, -0.0736,  0.1402]]), b:0.0
Epoch ( 100/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-0.0
Epoch ( 200/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-1e-07
Epoch ( 300/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-1e-07
Epoch ( 400/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-2e-07
Epoch ( 500/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-2e-07
Epoch ( 600/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-3e-07
Epoch ( 700/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-3e-07
Epoch ( 800/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-4e-07
Epoch ( 900/1000) cost: 0.006376, W: tensor([[-0.0906, -0.0778,  0.1683]]), b:-4e-07
W: tensor([[-0.0906, -0.0778,  0.1683]])
b: tensor([-4.4441e-07])
result : 
tensor([[0.3023, 0.3062, 0.3916],
        [0.2700, 0.2770, 0.4531],
        [0.2

In [11]:
# Chat GPT 가 내놓은 대답....
# 미친 거 같음 

import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(1, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Prepare the data
x_data = [[1], [2], [3], [11], [12], [13], [21], [22], [23]]
y_data = [0, 0, 0, 1, 1, 1, 2, 2, 2]

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

# Define the model, loss function, and optimizer
model = MyModel()

## 요 오차 구해주는 함수는 오차 평균값으로 계산해줌
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
num_epochs = 1000
batch_size = 9
for epoch in range(num_epochs):
    # Shuffle the data before each epoch
    indices = torch.randperm(x_train.shape[0])
    x_train = x_train[indices]
    y_train = y_train[indices]

    for i in range(0, len(x_data), batch_size):
        # Forward pass
        inputs = x_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.4f}")

# Evaluate the model
# model.eval() 효과

# 1. dropout 이 임의로 몇개 input 0으로 바꾸는데 이과정 없앰 
# 2. batch normalization

model.eval()
with torch.no_grad():
    outputs = model(x_train)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_train).sum().item() / len(y_train)
    print(f"Accuracy: {accuracy:.2f}")

Epoch 0/1000, Loss: 1.1870
Epoch 100/1000, Loss: 0.2698
Epoch 200/1000, Loss: 0.0833
Epoch 300/1000, Loss: 0.0366
Epoch 400/1000, Loss: 0.0219
Epoch 500/1000, Loss: 0.0152
Epoch 600/1000, Loss: 0.0113
Epoch 700/1000, Loss: 0.0088
Epoch 800/1000, Loss: 0.0071
Epoch 900/1000, Loss: 0.0059
Accuracy: 1.00
